In [1]:
!pip install pyngrok

In [2]:
import pandas as pd

data = pd.read_csv('/kaggle/input/heart-disease-health-indicators-dataset/heart_disease_health_indicators_BRFSS2015.csv')

data = data.drop(columns=['Stroke', 'DiffWalk', 'Education', 'Income'])

print(data.isnull().sum())
data = data.dropna()

HeartDiseaseorAttack    0
HighBP                  0
HighChol                0
CholCheck               0
BMI                     0
Smoker                  0
Diabetes                0
PhysActivity            0
Fruits                  0
Veggies                 0
HvyAlcoholConsump       0
AnyHealthcare           0
NoDocbcCost             0
GenHlth                 0
MentHlth                0
PhysHlth                0
Sex                     0
Age                     0
dtype: int64


In [3]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
for column in ['Sex', 'HighBP', 'HighChol', 'CholCheck', 'Smoker',
                'Diabetes', 'PhysActivity', 'Fruits', 'Veggies',
                'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost',
                'GenHlth']:
    data[column] = label_encoder.fit_transform(data[column])

In [4]:
X = data.drop(columns=['HeartDiseaseorAttack'])
y = data['HeartDiseaseorAttack']

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
import joblib

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
joblib.dump(model, 'random_forest_model.pkl')
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))
print('AUC-ROC Score:', roc_auc_score(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.92      0.97      0.94     69007
         1.0       0.35      0.13      0.19      7097

    accuracy                           0.90     76104
   macro avg       0.63      0.55      0.57     76104
weighted avg       0.86      0.90      0.87     76104

AUC-ROC Score: 0.5530666176635178


In [ ]:
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel
import joblib
import pandas as pd
from fastapi.middleware.cors import CORSMiddleware
from pyngrok import ngrok
import nest_asyncio

nest_asyncio.apply()

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

model = joblib.load('/kaggle/working/random_forest_model.pkl')

class HeartDiseaseInput(BaseModel):
    age: int
    sex: int
    high_bp: int
    high_chol: int
    chol_check: int
    bmi: float
    smoker: int
    diabetes: int
    phys_activity: int
    fruits: int
    veggies: int
    hvy_alcohol_consump: int
    any_healthcare: int
    no_doc_bc_cost: int
    gen_hlth: int
    ment_hlth: int
    phys_hlth: int

@app.get("/")
def read_root():
    return {"Message": "test"}

@app.post("/predict/")
def predict_heart_disease(data: HeartDiseaseInput):
    input_data = pd.DataFrame([data.dict()])
    prediction = model.predict(input_data)
    return {"prediction": int(prediction[0])}

if __name__ == "__main__":
    ngrok.set_auth_token("2lEbQnS7BNFOEssqkGSvAlc0abB_6ztWSbH8vg8Zhm6cVRzQy")
    public_url = ngrok.connect(8000)
    print(f"Public URL: {public_url}")
    uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [36]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "https://b0d3-34-82-115-196.ngrok-free.app" -> "http://localhost:8000"


In [ ]:
joblib.dump(model, '/kaggle/working/heart_disease_model.pkl')
joblib.dump(label_encoder, '/kaggle/working/label_encoder.pkl')